In [1]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.user_credential import UserCredential
from io import BytesIO
import pyodbc
import pandas as pd
import re
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
import pandas as pd
from io import StringIO
from office365.runtime.auth.user_credential import UserCredential
import smtplib
from email.message import EmailMessage
import os
from datetime import datetime, timedelta


In [2]:
# SharePoint site and authentication details
site_url = "https://lkceleste.sharepoint.com/sites/Stock"
username = "celestebi@celeste.lk"
password = "HelloData@24"

# File URLs for MWH and PWH
file_urls = [
    "/sites/Stock/Shared Documents/Transfer-Outs Request/dont_touch/merged_data.xlsx"
]

# Authenticate SharePoint context
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Initialize empty DataFrames for MWH and PWH
merged_data = None

# Read each file and convert to pandas DataFrame
for file_url in file_urls:
    response = File.open_binary(ctx, file_url)
    excel_data = BytesIO(response.content)
    
    # Read all sheets into a dictionary of DataFrames
    merged_data = pd.read_excel(excel_data, sheet_name=None)

# Check if the file was loaded correctly and print the available sheet names
if merged_data is not None:
    print("Available sheets in merged_data Replenishment:")
    print(merged_data.keys())  # List all sheet names in MWH
else:
    print("Error: merged_data file could not be loaded or is empty.")

merged_data_name = 'Merged_Data'  # Adjust this if necessary
if merged_data_name  in merged_data.keys():
    merged_data = merged_data[merged_data_name ]
    
    # Display the first few rows of the sheet
    print("merged_data Replenishment Sheet Data:")
    print(merged_data.head())
else:
    print(f"Sheet '{merged_data_name}' not found in merged_data_name.")

Available sheets in merged_data Replenishment:
dict_keys(['Merged_Data'])
merged_data Replenishment Sheet Data:
   PRODUCT_CODE  KOTTE_STOCK  COLOMBO_STOCK  ATTIDIYA_STOCK  PWH_STOCK  \
0           1.0         12.0           17.0             5.0        0.0   
1           2.0         23.0           19.0            27.0        0.0   
2           3.0         27.0           30.0            22.0        0.0   
3           7.0         28.0           17.0            21.0        0.0   
4          10.0          2.0            0.0             2.0        0.0   

   MWH_STOCK  Items Code                                   Item Description  \
0       18.0           1  ADULT CHICKEN VEGETABLE 3KG - PEDIGREE - SKU 0...   
1       10.0           2       ADULT OCEAN FISH 480G - WHISKAS - SKU 000002   
2        0.0           3             ADULT TUNA 480G - WHISKAS - SKU 000003   
3        0.0           7  ANLENE LOW FAT MILK POWDER 400G - ANCHOR - SKU...   
4        0.0          10  CELESTE X-MAS APPLE CI

In [3]:
# SharePoint site and authentication details
site_url = "https://lkceleste.sharepoint.com/sites/Stock"
username = "celestebi@celeste.lk"
password = "HelloData@24"

# File URLs for MWH and PWH
file_urls = [
    "/sites/Stock/Shared Documents/Transfer-Outs Request/MWH Replenishment.xlsx"
]

# Authenticate SharePoint context
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Initialize empty DataFrames for MWH and PWH
df_Pwh = None

# Read each file and convert to pandas DataFrame
for file_url in file_urls:
    response = File.open_binary(ctx, file_url)
    excel_data = BytesIO(response.content)
    
    # Read all sheets into a dictionary of DataFrames
    df_Pwh = pd.read_excel(excel_data, sheet_name=None)

# Check if the file was loaded correctly and print the available sheet names
if df_Pwh is not None:
    print("Available sheets in MWH Replenishment:")
    print(df_Pwh.keys())  # List all sheet names in MWH
else:
    print("Error: MWH file could not be loaded or is empty.")

# Assuming there's a sheet called 'MWH', if it exists, we access it
Pwh_sheet_name = 'MWH'  # Adjust this if necessary
if Pwh_sheet_name in df_Pwh.keys():
    Pwh_sheet1_df = df_Pwh[Pwh_sheet_name]
    
    # Display the first few rows of the sheet
    print("MWH Replenishment Sheet Data:")
   # print(Pwh_sheet1_df.head())
else:
    print(f"Sheet '{Pwh_sheet_name}' not found in PWH Replenishment.")

Available sheets in MWH Replenishment:
dict_keys(['Summary', 'MWH'])
MWH Replenishment Sheet Data:


In [4]:
print(Pwh_sheet1_df.columns)


Index(['ITEM CODE', 'ITEM DESCRPTION', 'SUPPLIER', 'DEPARTMENT',
       'SUB DEPARTMENT', 'WAREHOUSE', 'BARCODE', 'SELLING PRICE',
       'SKU Category', 'Priority', 'Col 6', 'Att', 'Ekotte', 'WAREHOUSE SIH',
       'Final Replenishment Qty Col', 'Final Replenishment Qty Att',
       'Final Replenishment Qty Ekotte', 'Total', 'Balance', 'Case Config'],
      dtype='object')


In [5]:
# Drop specified columns from the DataFrame
Pwh_sheet1_df.drop(columns=['SKU Category', 'SUPPLIER', 'DEPARTMENT', 'SUB DEPARTMENT',  'Priority', 'Col 6', 'Att', 'Ekotte','Balance','Case Qty'], inplace=True, errors='ignore')
Pwh_sheet1_df

,ITEM CODE,ITEM DESCRPTION,WAREHOUSE,BARCODE,SELLING PRICE,WAREHOUSE SIH,Final Replenishment Qty Col,Final Replenishment Qty Att,Final Replenishment Qty Ekotte,Total,Case Config
0,91,3 IN 1 NESCAFE ORIGINAL (18G X 25STICKS) - SKU...,MWH,0,2750,16,2,0,0,2,1
1,6987,ALOE KOMA ORIGINAL 200ML - NATIVE HARBALS - SK...,MWH,4796009050523,160,19,0,0,4,4,1
2,3433,APPLE JUICE 1L - MY JUICEE - SKU 003433,MWH,4792116204033,600,108,24,0,12,36,12
3,4153,APPLE NECTAR 200ML - KIST - SKU 004153,MWH,4792143131524,140,12,6,0,0,6,24
4,2472,AQUAFRESH 1.5L CLASSIC - SKU 002472,MWH,4796000210049,185,375,48,0,0,48,12
...,...,...,...,...,...,...,...,...,...,...,...
375,1201,PANADOL LIQUID FOR CHILDREN 100ML - SKU 001201,MWH,4792099010201,370,40,18,0,18,36,36
376,7092,STUDDED CONDOM - SATHUTA - SKU 007092,MWH,,150,24,0,0,24,24,24
377,6586,SWASTHA THRIPHALA 30 TABLETS - LINK - SKU 006586,MWH,4792022052117,360,14,0,0,2,2,1
378,6900,ZINC OXIDE PLASTER (1.25CMX1Y) - SOFTA - SKU -...,MWH,0,55,10,0,0,3,3,1


In [6]:
import pandas as pd


# Perform the merge
merged_df = pd.merge(merged_data, Pwh_sheet1_df, left_on='PRODUCT_CODE', right_on='ITEM CODE', how='left')
# Replace NaN values with 0
merged_df = merged_df.fillna(0)
# Display merged DataFrame
merged_df['SKU Category'] = merged_df['SKU Category'].str.upper()

In [7]:
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import pandas as pd
from io import BytesIO

# SharePoint site and authentication details
site_url = "https://lkceleste.sharepoint.com/sites/Stock"
username = "celestebi@celeste.lk"
password = "HelloData@24"

# Authenticate SharePoint context
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Example: Create a DataFrame `merge_df`
# Replace this with your actual DataFrame creation or loading logic


# Create a BytesIO buffer to hold the CSV data
csv_buffer = BytesIO()


# Save the merged DataFrame as a CSV in the buffer
merged_df.to_csv(csv_buffer, index=False)  # Set index=True if you want to include the index

# Reset buffer position to the beginning
csv_buffer.seek(0)

# Specify the target folder and file name for the CSV upload
target_folder_url = "/sites/Stock/Shared Documents/Transfer-Outs Request/dont_touch"
target_file_name = "MWH_merged_data_before.csv"  # Name of the CSV file to be uploaded
target_folder = ctx.web.get_folder_by_server_relative_url(target_folder_url)

# Check if the file already exists and delete it
try:
    existing_file = target_folder.files.get_by_url(target_file_name)
    existing_file.delete_object().execute_query()
    print(f"Existing file '{target_file_name}' deleted.")
except Exception as e:
    print(f"No existing file to delete: {e}")

# Upload the new CSV file
target_file = target_folder.upload_file(target_file_name, csv_buffer).execute_query()

# Print the server-relative URL of the uploaded file
print(f"File '{target_file.serverRelativeUrl}' has been uploaded.")


Existing file 'MWH_merged_data_before.csv' deleted.
File '/sites/Stock/Shared Documents/Transfer-Outs Request/dont_touch/MWH_merged_data_before.csv' has been uploaded.


In [8]:
print(merged_df.columns)

Index(['PRODUCT_CODE', 'KOTTE_STOCK', 'COLOMBO_STOCK', 'ATTIDIYA_STOCK',
       'PWH_STOCK', 'MWH_STOCK', 'Items Code', 'Item Description', 'Supplier',
       'Department', 'Warehouse', 'SKU Category', 'Col Active / Inactive ',
       'Att Active / Inactive ', 'Kotte Active / Inactive ', 'ITEM CODE',
       'ITEM DESCRPTION', 'WAREHOUSE', 'BARCODE', 'SELLING PRICE',
       'WAREHOUSE SIH', 'Final Replenishment Qty Col',
       'Final Replenishment Qty Att', 'Final Replenishment Qty Ekotte',
       'Total', 'Case Config'],
      dtype='object')


In [9]:
import pandas as pd
from io import BytesIO
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential

# SharePoint site and authentication details
site_url = "https://lkceleste.sharepoint.com/sites/Stock"
username = "celestebi@celeste.lk"
password = "HelloData@24"

# Authenticate SharePoint context
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Assuming merged_df is already defined and contains the necessary data

# Filter DataFrame for Warehouse 'PWH'
filtered_df = merged_df[merged_df['Warehouse'] == 'MWH']

# PIVOT 1: Active Product Counts
def get_active_counts(df, active_col):
    return df[df[active_col].str.strip() == 'A'].groupby('SKU Category')['PRODUCT_CODE'].count()

colombo_counts = get_active_counts(filtered_df, 'Col Active / Inactive ')
attidiya_counts = get_active_counts(filtered_df,  'Att Active / Inactive ')
kotte_counts = get_active_counts(filtered_df, 'Kotte Active / Inactive ')

pivot_table_1 = pd.DataFrame({
    'COLOMBO': colombo_counts,
    'ATTIDIYA': attidiya_counts,
    'KOTTE': kotte_counts
}).fillna(0).astype(int)

# Reset index to include SKU Category as a column
pivot_table_1 = pivot_table_1.reset_index()

print("BEFORE Count of Active Products for MWH by SKU Category and Stock Location")
print(pivot_table_1)

# PIVOT 2: Out-of-Stock Counts and Percentages
def get_out_of_stock_counts(df, stock_col, active_col):
    return df[(df[active_col].str.strip() == 'A') & (df[stock_col] <= 0)].groupby('SKU Category')['PRODUCT_CODE'].count()

colombo_oos = get_out_of_stock_counts(filtered_df, 'COLOMBO_STOCK', 'Col Active / Inactive ')
attidiya_oos = get_out_of_stock_counts(filtered_df, 'ATTIDIYA_STOCK', 'Att Active / Inactive ')
kotte_oos = get_out_of_stock_counts(filtered_df, 'KOTTE_STOCK', 'Kotte Active / Inactive ')

pivot_table_oos = pd.DataFrame({
    'COLOMBO': colombo_oos,
    'ATTIDIYA': attidiya_oos,
    'KOTTE': kotte_oos
}).fillna(0).astype(int)

oos_percentage = (pivot_table_oos / pivot_table_1.set_index('SKU Category').replace(0, float('nan'))) * 100
final_pivot_table_2 = pivot_table_oos.copy()
final_pivot_table_2['COLOMBO Percentage'] = oos_percentage['COLOMBO']
final_pivot_table_2['ATTIDIYA Percentage'] = oos_percentage['ATTIDIYA']
final_pivot_table_2['KOTTE Percentage'] = oos_percentage['KOTTE']

# Reset index to include SKU Category as a column
final_pivot_table_2 = final_pivot_table_2.reset_index()

print("\nBEFORE Count of Out-of-Stock Products and Percentages for MWH by SKU Category and Stock Location")
print(final_pivot_table_2)

# Function to upload a DataFrame as a CSV file to SharePoint
def upload_to_sharepoint(df, file_name, folder_url):
    # Create a BytesIO buffer to hold the CSV data
    csv_buffer = BytesIO()
    df.to_csv(csv_buffer, index=False)  # Save DataFrame to CSV in memory
    csv_buffer.seek(0)  # Reset buffer position to the beginning

    # Get the target folder
    target_folder = ctx.web.get_folder_by_server_relative_url(folder_url)

    # Check if the file already exists and delete it
    try:
        existing_file = target_folder.files.get_by_url(file_name)
        existing_file.delete_object().execute_query()
        print(f"Existing file '{file_name}' deleted.")
    except Exception as e:
        print(f"No existing file to delete: {e}")

    # Upload the new CSV file
    target_file = target_folder.upload_file(file_name, csv_buffer).execute_query()
    print(f"File '{target_file.serverRelativeUrl}' has been uploaded.")

# Specify the target folder for the CSV uploads
target_folder_url = "/sites/Stock/Shared Documents/Transfer-Outs Request/dont_touch"

# Upload each pivot table as a CSV file
upload_to_sharepoint(pivot_table_1, "MWH_active_counts.csv", target_folder_url)
upload_to_sharepoint(final_pivot_table_2, "MWH_out_of_stock_counts.csv", target_folder_url)

print("All pivot tables have been uploaded to SharePoint.")

BEFORE Count of Active Products for MWH by SKU Category and Stock Location
  SKU Category  COLOMBO  ATTIDIYA  KOTTE
0            A      673       667    673
1            B      569       547    569
2            C      931       779    905
3           NL      132       118    128

BEFORE Count of Out-of-Stock Products and Percentages for MWH by SKU Category and Stock Location
  SKU Category  COLOMBO  ATTIDIYA  KOTTE  COLOMBO Percentage  \
0            A      109       103    104           16.196137   
1            B      147       136    144           25.834798   
2            C      221       245    294           23.737916   
3           NL       58        64     65           43.939394   

   ATTIDIYA Percentage  KOTTE Percentage  
0            15.442279         15.453195  
1            24.862888         25.307557  
2            31.450578         32.486188  
3            54.237288         50.781250  
Existing file 'MWH_active_counts.csv' deleted.
File '/sites/Stock/Shared Documents/Tra